In [8]:
import torch
import model
from torchsummary import summary
mod = model.get_RepVgg_func_by_name('RepVgg-A0')().cuda()

x = torch.randn((1,3,224,224)).cuda()
mod = model.repVgg_model_convert(mod)
print(summary(mod, input_size=(3,224,224)))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 112, 112]           1,344
          Identity-2         [-1, 48, 112, 112]               0
              ReLU-3         [-1, 48, 112, 112]               0
       RepVggblock-4         [-1, 48, 112, 112]               0
            Conv2d-5           [-1, 48, 56, 56]          20,784
          Identity-6           [-1, 48, 56, 56]               0
              ReLU-7           [-1, 48, 56, 56]               0
       RepVggblock-8           [-1, 48, 56, 56]               0
            Conv2d-9           [-1, 48, 56, 56]          20,784
         Identity-10           [-1, 48, 56, 56]               0
             ReLU-11           [-1, 48, 56, 56]               0
      RepVggblock-12           [-1, 48, 56, 56]               0
           Conv2d-13           [-1, 96, 28, 28]          41,568
         Identity-14           [-1, 96,

In [22]:
import imp
from re import A
from cv2 import transform
from sklearn.utils import shuffle
import torch
import torchvision
import numpy as np
import os
import model
import wandb

batch_size = 8
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
'''
data
'''
train_root = "D:/deep_learing_work/data/cif10/train/raw-img"
test_root = "D:/deep_learing_work/data/cif10/test/raw-img"
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean = [0.485, 0.456,0.406],std = [0.229, 0.224, 0.225])
])
data_iter = torchvision.datasets.ImageFolder(root=train_root, transform=transform)
data_load = torch.utils.data.DataLoader(data_iter, batch_size = batch_size, shuffle = True)

test_iter = torchvision.datasets.ImageFolder(root=test_root, transform=transform)
test_load = torch.utils.data.DataLoader(test_iter, batch_size = batch_size, shuffle = False)

epochs = 20
lossf = torch.nn.CrossEntropyLoss()
lr = 1e-5
# wandb.init(entity='wtj', project='Repvgg object classifier')
# wandb.config = {
#     'lr':0.0001,
#     'epochs':20, 
#     'batch_size':8
# }

modelRepVgg = model.get_RepVgg_func_by_name('RepVgg-A0')(num_classes = 10).to(device)
modelRepVgg.train()
optimizer = torch.optim.SGD(modelRepVgg.parameters(), lr = lr)

for eopch in range(epochs):
    loss_arr = []
    for x_cpu, y_cpu in data_load:
        optimizer.zero_grad()
        x = x_cpu.to(device)
        y = y_cpu.to(device)
        y_p = modelRepVgg(x)
        train_loss = lossf(y_p, y).sum()
        train_loss.backward()
        loss_arr.append(train_loss.detach().cpu())
        optimizer.step()
    modelRepVgg.eval()
    with torch.no_grad():
        train_mean_loss = torch.tensor(loss_arr)
        correct = 0
        total = 0
        for x, y in test_load:
            x, y = x.to(device),y.to(device)
            y_p = modelRepVgg(x)
            y_p = y_p.cpu()
            y_p = torch.argmax(y_p, dim = 1)
            label = y.cpu()
            correct +=  (y_p == label).sum()
            total += batch_size
        test_acc = correct / total
        wandb.log({'train_loss':train_mean_loss,'test_acc:':test_acc})    
    modelRepVgg.train()
    model.save(modelRepVgg, './RepVgg.pt')
    



tensor(0, device='cuda:0')


KeyboardInterrupt: 